In [12]:
import tqdm 
import pandas as pd
import akshare as ak

def filter_shsz(v):
    code,name = v["code"],v["name"]
    if code.startswith("68") or code.startswith("8")or code.startswith("4"):
        return False
    elif "ST" in name or "退市" in name or "C" in name:
        return False
    elif code[0] not in ("0","3","6"):
        return False
    return True

def reform_shsz(code):
    if code.startswith("3") or code.startswith("00"):
        return "sz"+code
    elif code.startswith("6"):
        return "sh"+code
    return code
    
    
    
df = ak.stock_zh_a_spot_em()
df = df.rename(columns={"代码":"code","名称":"name"})
df = df[df.apply(filter_shsz,axis=1)]
df["code"] = df["code"].apply(reform_shsz)
cdf = df[["code","name"]]
lst = []
hdf = pd.read_csv("stock_hist.csv")
hs = set(hdf["code"].drop_duplicates())
cdf = cdf[~cdf.code.isin(hs)]
lst = [hdf]
bar = tqdm.tqdm(total =cdf.shape[0])

for i,(idx,v)in enumerate(cdf.iterrows()):
    bar.update()
    if i>1000:
        break
    try:
        tdf = ak.stock_zh_a_cdr_daily(symbol=v["code"], start_date="20241116")[["date","open","high","low","close","volume","amount"]]
        tdf[["code","name"]]=(v["code"],v["name"])
        lst.append(tdf)
    except:
        print(v)
if len(lst)>1:
    hdf = pd.concat(lst)
    hdf['code'] = hdf['code'].astype(str)
    hdf.to_csv("stock_hist.csv",index=False)



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 139/139 [01:12<00:00,  1.92it/s]

  2%|██▌                                                                                                                   | 3/139 [00:00<00:06, 20.74it/s]

code    sh603194
name        中力股份
Name: 571, dtype: object
code    sh603072
name        天和磁材
Name: 576, dtype: object



 24%|████████████████████████████▌                                                                                        | 34/139 [00:07<00:17,  5.91it/s]

code    sz301581
name        黄山谷捷
Name: 699, dtype: object
code    sz301458
name        钧崴电子
Name: 700, dtype: object



 68%|███████████████████████████████████████████████████████████████████████████████                                      | 94/139 [00:19<00:08,  5.01it/s]

code    sz001391
name         国货航
Name: 793, dtype: object



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 139/139 [00:32<00:00,  3.50it/s]

In [1]:
import akshare as ak
import datetime

def filter_shsz(v):
    code,name = v["code"],v["name"]
    if code.startswith("68") or code.startswith("8")or code.startswith("4"):
        return False
    elif "ST" in name or "退市" in name or "C" in name:
        return False
    elif code[0] not in ("0","3","6"):
        return False
    return True


def reform_shsz(code):
    if code.startswith("3") or code.startswith("00"):
        return "sz"+code
    elif code.startswith("6"):
        return "sh"+code
    return code
    
today = datetime.datetime.now()
cdf = ak.stock_zh_a_spot_em()
cdf = cdf.rename(columns={"代码":"code","名称":"name"})
cdf = cdf[cdf.apply(filter_shsz,axis=1)]
cdf["code"] = cdf["code"].apply(reform_shsz)
cdf = cdf.rename(columns={"今开":"open","最低":"low","最高":"high","最新价":"close",
                         "成交量":"volume","成交额":"amount"})
cdf["date"]=today.strftime('%Y-%m-%d')
cdf["volume"] = cdf["volume"]*150
cdf["amount"] = cdf["amount"]*1.5
cdf = cdf[["date","code","name","open","low","high","close","volume","amount"]]
# hdf = pd.read_csv("stock_hist.csv")
# df = pd.concat([hdf,cdf])
# cdf.to_csv("stock_today.csv",index=False)
cdf.to_csv("stock_today.csv",index=False)
cdf.head()

/Users/john/anaconda3/lib/python3.11/site-packages/akshare/__init__.py:2977: UserWarning: 为了支持更多特性，请将 Pandas 升级到 2.2.0 及以上版本！
  warnings.warn(


,date,code,name,open,low,high,close,volume,amount
0,2024-12-27,sz300141,和顺电气,9.23,9.23,11.14,11.14,83312700.0,9.180881e+08
1,2024-12-27,sz300599,雄塑科技,6.60,6.60,7.67,7.67,23268000.0,1.752030e+08
2,2024-12-27,sz300547,川环科技,24.41,24.41,24.41,24.41,25273500.0,6.169261e+08
3,2024-12-27,sz301315,威士顿,54.08,54.08,65.82,65.82,32515950.0,2.017005e+09
5,2024-12-27,sz300189,神农种业,3.49,3.44,4.12,4.06,321378750.0,1.235812e+09


In [2]:
import pandas as pd
import tqdm
today = datetime.datetime.now()
bias = 0
cdate, pdate = today+datetime.timedelta(days=-bias),today+datetime.timedelta(days=-bias-1)
cdate,pdate = cdate.strftime('%Y-%m-%d'),pdate.strftime('%Y-%m-%d')
df = pd.read_csv(f"daily/stock_{pdate}.csv")
cdf = pd.read_csv("stock_today.csv")
df = pd.concat([df,cdf])
df.to_csv(f"daily/stock_{cdate}.csv",index=False)
cdf.to_csv(f"daily/stock_daily_{cdate}.csv",index=False)